In [ ]:
pip install diffusers --upgrade

In [ ]:
pip install invisible_watermark transformers accelerate safetensors

In [ ]:
pip install torchvision

In [ ]:
pip install rembg

In [ ]:
pip install onnxruntime

In [ ]:
!pip install open3d

In [ ]:
import torch

In [ ]:
from diffusers import DiffusionPipeline

In [ ]:
#Using Stable diffusion from Hugging Face for creating images from the text input
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

In [ ]:
#choose either Text or Image as Inputprint("What do you want to enter Text or Image")
choice=int(input("Press 1. Text\n      2. Image\n"))
print(choice)

In [ ]:
from google.colab import files
from PIL import Image
if choice==1:
  prompt=input("enter text ")
  images = pipe(prompt=prompt,height=1024,width=1024).images[0]
else:
  uploaded = files.upload()
  for filename in uploaded.keys():
    print("Uploaded file name:", filename)
  images = Image.open(filename)


In [ ]:
images.save("fullimage.png")

In [ ]:
import rembg

In [ ]:
from rembg import remove
from PIL import Image

In [ ]:
input_image=Image.open('/content/fullimage.png')
output=remove(input_image)
output.save('output.png')


In [ ]:
from transformers import pipeline
from PIL import Image
import requests

# load pipe
depth_estimator = pipeline(task="depth-estimation", model="Intel/zoedepth-nyu-kitti")

# load image
image_path = "/content/output.png"  # Change to your imageblob:https://colab.research.google.com/cf3444c0-62d6-434e-897a-0212bdc875ec path
image = Image.open(image_path).convert("RGB")


# inference
outputs = depth_estimator(image)
#depth = outputs.depth


In [ ]:
depth = outputs['depth']

# Save or display the depth map
depth.save("/content/depth_map.png")
depth.show()

In [ ]:
import open3d as o3d

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load depth image (grayscale PNG) and RGB image
depth = np.array(Image.open("/content/depth_map.png").convert("L"))  # or 'I' if 16-bit
rgb = np.array(Image.open("/content/fullimage.png"))

In [ ]:
# Normalize and convert depth to Open3D format
depth_o3d = o3d.geometry.Image(depth.astype(np.uint16))  # use uint16 for compatibility
color_o3d = o3d.geometry.Image(rgb)

In [ ]:
# Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d, depth_scale=255.0, convert_rgb_to_intensity=False
)

In [ ]:
# Camera intrinsics (modify if needed)
width, height = depth.shape[1], depth.shape[0]
fx = fy = 500.0
cx = width / 2
cy = height / 2
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

In [ ]:
# Generate point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)

# Convert to numpy for plotting
points = np.asarray(pcd.points)

In [ ]:
# Check if points exist
if points.size == 0:
    print("Point cloud is empty. Check depth map and intrinsics.")
else:
    o3d.io.write_point_cloud("point_cloud.ply", pcd)

    # Ploting 2D projection
    plt.figure(figsize=(10, 6))
    plt.scatter(points[:, 0], points[:, 1], c=points[:, 2], cmap='plasma', s=0.5)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("2D Point Cloud Projection (X vs Y)")
    plt.colorbar(label="Z Depth")
    plt.grid(True)
    plt.show()


In [ ]:
color_raw=o3d.io.read_image("/content/fullimage.png")
depth_raw=o3d.io.read_image("/content/depth_map.png")
rgbd_image=o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw,depth_raw)
print(rgbd_image)

In [ ]:
plt.subplot(1,2,1)
plt.title("Greyscale Image")
plt.imshow(rgbd_image.color)
plt.subplot(1,2,2)
plt.title("Depth Image")
plt.imshow(rgbd_image.depth)
plt.show()

In [ ]:
import open3d as o3d
import os
import shutil

check = 0
final_voxel = None

#Compressing the .ply file if it exceeds 100MB because most of PLY viewer does not support viewing more than 100MB data
def compress_ply_if_large(ply_path, output_dir="/content", voxel_sizes=[0.01, 0.005, 0.002], size_limit_mb=100):
    global check, final_voxel

    file_size_mb = os.path.getsize(ply_path) / (1024 * 1024)

    if file_size_mb <= size_limit_mb:
        print(f"File size is {file_size_mb:.2f} MB, no compression needed.")
        return ply_path

    check = 1
    print(f"File size is {file_size_mb:.2f} MB, compressing...")

    pcd = o3d.io.read_point_cloud(ply_path)
    original_points = len(pcd.points)

    for voxel_size in voxel_sizes:
        print(f"Trying voxel size: {voxel_size}")
        downsampled = pcd.voxel_down_sample(voxel_size)
        points_left = len(downsampled.points)
        print(f"Points remaining: {points_left}/{original_points}")

        if points_left > 0.3 * original_points:
            final_voxel = voxel_size
            output_path = os.path.join(output_dir, f"point_cloud_compressed_{voxel_size}.ply")
            o3d.io.write_point_cloud(output_path, downsampled)
            print(f"Saved compressed PLY to: {output_path}")
            return output_path

    print("All voxel sizes removed too many points. Skipping compression.")
    return ply_path

#Returning obj file
def change_extension(ply_path):
    if os.path.exists(ply_path):
        obj_path = os.path.splitext(ply_path)[0] + '.obj'
        shutil.copy(ply_path, obj_path)
        print(f"File copied and renamed to: {obj_path}")
    else:
        print(f"Error: {ply_path} does not exist.")

#calling the methods
compressed_path = compress_ply_if_large("/content/point_cloud.ply")
change_extension(compressed_path)
